In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import geopandas as gpd

In [22]:
data = pd.read_csv('dft_statistics_collision_and_casualty_last_5_years.csv', low_memory = False)

In [23]:
data.rename(columns={'accident_year_x': 'accident_year', 'accident_reference_x': 'accident_reference'}, inplace=True)
data.drop(labels=['accident_year_y', 'accident_reference_y', 'police_force'], axis=1, inplace=True)

In [24]:
data.date = pd.to_datetime(data.date, format ='%d/%m/%Y')
data.time = pd.to_datetime(data.time, format='%H:%M')

In [26]:
data.dtypes

accident_index                                         object
accident_year                                           int64
accident_reference                                     object
location_easting_osgr                                 float64
location_northing_osgr                                float64
longitude                                             float64
latitude                                              float64
accident_severity                                      object
number_of_vehicles                                      int64
number_of_casualties                                    int64
date                                           datetime64[ns]
day_of_week                                            object
time                                           datetime64[ns]
local_authority_district                               object
local_authority_ons_district                           object
local_authority_highway                                object
first_ro

In [41]:
data.isna().sum()

accident_index                                     0
accident_year                                      0
accident_reference                                 0
location_easting_osgr                              1
location_northing_osgr                             1
longitude                                          1
latitude                                           1
accident_severity                                  0
number_of_vehicles                                 0
number_of_casualties                               0
date                                               0
day_of_week                                        0
time                                               0
local_authority_district                        5411
local_authority_ons_district                       0
local_authority_highway                            0
first_road_class                                   0
first_road_number                               8067
road_type                                     

In [33]:
test = gpd.GeoDataFrame(data, geometry = gpd.points_from_xy(data.longitude, data.latitude))
test.crs = {'init': 'epsg:27700'}
